# Verbatim mode

In [1]:
from braket.circuits import Circuit, Observable, Gate, Noise
from braket.aws import AwsDevice
import numpy as np
import datetime

In [2]:
now = datetime.datetime.now().strftime('%y%m%d%H%M%S')
my_bucket = "amazon-braket-hardware-testing" # the name of the bucket
my_prefix = "oqc-verbatim-" + now # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

# device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-11")
device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

In [3]:
# Available gates
for gate in device.properties.paradigm.nativeGateSet:
    print(gate)

rx
rz
cz
cphaseshift
xy


In [4]:
identity = Circuit().rz(0,np.pi).rz(0,np.pi)
circ = Circuit().add_verbatim_box(identity)
circ = circ.z(0)
print(circ)

T  : |      0      |   1    |   2    |     3     |
                                                  
q0 : -StartVerbatim-Rx(3.14)-Rx(3.14)-EndVerbatim-

T  : |      0      |   1    |   2    |     3     |


### See result 

In [5]:
# For Rigetti's device
# result = device.run(circ, s3_folder, shots=100, disable_qubit_rewiring=True).result()
# compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
# print(compiled_program)
# print(result.measurement_counts)

DECLARE ro BIT[1]
PRAGMA INITIAL_REWIRING "NAIVE"
RX(pi) 0
RX(pi) 0
MEASURE 0 ro[0]

Counter({'0': 99, '1': 1})


In [ ]:
# OQC
# reference: https://github.com/aws/amazon-braket-schemas-python-staging/blob/main/src/braket/task_result/oqc_metadata_v1.py
result = device.run(circ, s3_folder, shots=100, disable_qubit_rewiring=True).result()
compiled_program = result.additional_metadata.OqcMetadata.executedProgram
print(compiled_program)
print(result.measurement_counts)